In [ ]:
from requests import Session
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '639f7cca301840d584c93fe3387437b7',
}

session = Session()
session.headers.update(headers)

try:
    # verify=False 
    response = session.get(url, params=parameters, verify=False)
    data = json.loads(response.text)
    
    
    if data['status']['error_code'] == 0:
        for coin in data['data']:
            print(f"Coin: {coin['name']} | Price: {coin['quote']['USD']['price']:.2f}")
    else:
        print("Hata mesajı:", data['status']['error_message'])

except Exception as e:
    print("error", e)

#NOTE:
# I had to go in and put "jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10"
# Into the Anaconda Prompt to change this to allow to pull data

In [ ]:
print(data)

In [ ]:
type(data)

In [ ]:
import pandas as pd

#This allows you to see all the columns, not just like 15
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)

In [ ]:
#This normalizes the data and makes it all pretty in a dataframe
df=pd.json_normalize(data['data'])

In [ ]:
df['timestamp']=pd.to_datetime('now')
df

In [ ]:
import pandas as pd
import json
import os
import urllib3
from requests import Session
from time import sleep
from datetime import datetime

# Suppress SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# GLOBAL dataframe for memory storage
df = pd.DataFrame()

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

parameters = {
    'start': '1',
    'limit': '15',
    'convert': 'USD'
}

# Fetching the key from environment variables
headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': os.getenv('CMC_API_KEY')
}

session = Session()
session.headers.update(headers)

def api_runner():
    global df
    
    # API request with SSL verification disabled for local stability
    response = session.get(url, params=parameters, verify=False)
    data = response.json()
    
    if 'data' not in data:
        print("No data returned:", data)
        return
    
    # Normalize current batch of data
    df2 = pd.json_normalize(data['data'])
    df2['Timestamp'] = datetime.now()
    
    # Keep adding to the global dataframe in memory
    df = pd.concat([df, df2], ignore_index=True)
    
    csv_path = r'C:\Users\CemK\Documents\anaconda_projects\Python Scripts\API.csv'
    
    # CSV persistence logic: 
    # Use df2 for append mode to prevent duplicating the entire memory buffer into the file
    if not os.path.isfile(csv_path):
        df2.to_csv(csv_path, header='column_names')
    else: 
        df2.to_csv(csv_path, mode='a', header=False)

    print(f"API Runner completed. Current memory rows: {len(df)}")


# MAIN LOOP
for i in range(5):
    api_runner()
    sleep(60)

print("Finished")

In [ ]:
df3=pd.read_csv(r'C:\Users\CemK\Documents\anaconda_projects\Python Scripts\API.csv')
df3

In [ ]:
# the scientific notation
pd.set_option('display.float_format', lambda x:'%.5f' % x)

In [ ]:
df

In [ ]:
# Now let's look at the coin trends over time
df3=df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean() 
df3

In [ ]:
df4=df3.stack()
df4

In [ ]:
type(df4)

In [ ]:
df5=df4.to_frame(name='values')
df5

In [ ]:
type(df5)

In [ ]:
df5.count()

In [ ]:

Index=pd.Index(range(90))


# Set the above DataFrame index object as the index
# using set_index() function
df6=df5.reset_index()
df6

# If it only has the index and values try doing reset_index like "df5.reset_index()"

In [ ]:
# Change the column name

df7=df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
df7['percent_change']=df7['percent_change'].replace(['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h', '7d', '30d', '60d', '90d'])
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 

sns.catplot(x='percent_change', y='values', hue='name', data=df7, kind='point')

In [ ]:
# Now to do something much simpler
# we are going to create a dataframe with the columns we want

df8 = df[['name','quote.USD.price','Timestamp']]
df8 = df8.query("name == 'Bitcoin'")
df8

In [ ]:
sns.set_theme(style="darkgrid")

sns.lineplot(x='Timestamp', y='quote.USD.price', data=df8)